# Introduction

...

# Data manipulation

In [36]:
import pandas as pd
import numpy as np
import os


Since the structure of the data folder is complex (some countries include multiple leagues, some not), we somehow need to iterate over all folders and all end files.

## Indexing all files

In [37]:
# We will save the root directory of the data
root_dir = "raw_data"  
file_paths = []

# Now we can walk through the directory structure
for dirpath, dirnames, filenames in os.walk(root_dir):

    # Check each file in the current directory
    for filename in filenames:

        # Filter for CSV files
        if filename.endswith(".csv"):
            file_path = os.path.join(dirpath, filename)

            # Keep track of all file paths
            file_paths.append(file_path)
            
            
# Let's look at the first file we found and see if we can load it
data = pd.read_csv(file_paths[0])
data.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,B1,26/07/2019,19:30,Genk,Kortrijk,2,1,H,0,1,...,2.53,-1.25,1.98,1.88,2.01,1.88,2.05,1.96,1.96,1.87
1,B1,27/07/2019,17:00,Cercle Brugge,Standard,0,2,A,0,0,...,2.24,0.50,1.88,1.98,1.91,1.99,1.95,2.02,1.89,1.94
2,B1,27/07/2019,19:00,St Truiden,Mouscron,0,1,A,0,1,...,2.10,-0.25,1.83,2.02,1.87,2.03,1.88,2.07,1.83,2.02
3,B1,27/07/2019,19:00,Waregem,Mechelen,0,2,A,0,1,...,2.28,0.25,1.72,2.07,1.75,2.17,1.80,2.28,1.72,2.15
4,B1,27/07/2019,19:30,Waasland-Beveren,Club Brugge,1,3,A,1,1,...,2.73,1.50,2.20,1.70,2.19,1.74,2.25,1.83,2.11,1.74


## Creating the limited dataset

Ok, it looks like the walk worked and now we have a list of all paths to the csv files. Our first task here is to complete the baseline limited dataset from all files.

In [38]:
# Let's initialize an empty DataFrame to hold our limited data
limited_data = pd.DataFrame()

# Now let's create a set of features that appear in all files (we will use these later)
shared_features = set()

for file_path in file_paths:

    # Read the CSV file into a DataFrame
    temp_df = pd.read_csv(file_path)

    # Extract all feature names (column names)
    all_features = temp_df.columns.tolist()

    # Update the shared_features set with features from the current file
    shared_features.update(all_features)
    shared_features = shared_features.intersection(set(all_features))


    base_features = ["Div", "Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR"]

    # Check if temp_df contains all the base features
    if all(feature in all_features for feature in base_features):
        temp_df = temp_df[base_features]
        limited_data = pd.concat([limited_data, temp_df], ignore_index=True)
    else:
        missing_features = [feature for feature in base_features if feature not in all_features]
        print(f"File: {file_path} is missing features: {missing_features}")

limited_data.shape

(42593, 8)

At some point, we will have to decide on how to encode categorical features, especially the teams. 

## Creating the full dataset

In [39]:
full_data = pd.DataFrame()

for file_path in file_paths:

    # Read the CSV file into a DataFrame
    temp_df = pd.read_csv(file_path)

    # Extract the division name for potential future use
    division = temp_df["Div"].iloc[0]

    # Extract all feature names (column names)
    all_features = temp_df.columns.tolist()
    
    # Check if temp_df contains any extra features, other than the shared features
    if any(feature not in all_features and feature not in shared_features for feature in all_features):
        print(f"File: {file_path} has no extra features.")
    else:
        extra_features = [feature for feature in all_features if feature not in shared_features]
        print(f"File: {file_path} has extra features: {extra_features}")
    full_data = pd.concat([full_data, temp_df], ignore_index=True)

full_data.shape

File: raw_data\belgium\1\1920.csv has extra features: ['IWH', 'IWD', 'IWA', 'VCH', 'VCD', 'VCA', 'IWCH', 'IWCD', 'IWCA', 'VCCH', 'VCCD', 'VCCA']
File: raw_data\belgium\1\2021.csv has extra features: ['IWH', 'IWD', 'IWA', 'VCH', 'VCD', 'VCA', 'IWCH', 'IWCD', 'IWCA', 'VCCH', 'VCCD', 'VCCA']
File: raw_data\belgium\1\2122.csv has extra features: ['IWH', 'IWD', 'IWA', 'VCH', 'VCD', 'VCA', 'IWCH', 'IWCD', 'IWCA', 'VCCH', 'VCCD', 'VCCA', 'Unnamed: 105']
File: raw_data\belgium\1\2223.csv has extra features: ['IWH', 'IWD', 'IWA', 'VCH', 'VCD', 'VCA', 'IWCH', 'IWCD', 'IWCA', 'VCCH', 'VCCD', 'VCCA']
File: raw_data\belgium\1\2324.csv has extra features: ['IWH', 'IWD', 'IWA', 'VCH', 'VCD', 'VCA', 'IWCH', 'IWCD', 'IWCA', 'VCCH', 'VCCD', 'VCCA']
File: raw_data\belgium\1\2425.csv has extra features: ['Unnamed: 119', 'Unnamed: 120']
File: raw_data\england\0\1920.csv has extra features: ['Referee', 'IWH', 'IWD', 'IWA', 'VCH', 'VCD', 'VCA', 'IWCH', 'IWCD', 'IWCA', 'VCCH', 'VCCD', 'VCCA']
File: raw_data\e

(42593, 137)

## First thoughts on encoding

In [40]:
# Let's see how many unique teams we have in the dataset
union_teams = set(limited_data['HomeTeam']).union(set(limited_data['AwayTeam']))
# Sets only allow unique values and union combines both sets, so this gives us all unique teams

print(f"Total unique teams: {len(union_teams)}")

Total unique teams: 478


There are 478 teams in total in our dataset, which is not a small number. One-hot encoding both columns would push our dataset to almost a 1000 features and would make it very sparse. If we decide to use target encoding, there is a pretty good chance that some new teams might appear in the test set, so we might have to encode before splitting into the train and test sets. This is definitely a risk of data leakage.

## Enhancing the limited dataset

In [41]:
limited_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42593 entries, 0 to 42592
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Div       42593 non-null  object
 1   Date      42593 non-null  object
 2   Time      42593 non-null  object
 3   HomeTeam  42593 non-null  object
 4   AwayTeam  42593 non-null  object
 5   FTHG      42593 non-null  int64 
 6   FTAG      42593 non-null  int64 
 7   FTR       42593 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.6+ MB


In [44]:
limited_data["Date"] = pd.to_datetime(limited_data["Date"], format="%d/%m/%Y")
limited_data["Time"] = pd.to_datetime(limited_data["Time"], format="%H:%M").dt.time
limited_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42593 entries, 0 to 42592
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Div       42593 non-null  object        
 1   Date      42593 non-null  datetime64[ns]
 2   Time      42593 non-null  object        
 3   HomeTeam  42593 non-null  object        
 4   AwayTeam  42593 non-null  object        
 5   FTHG      42593 non-null  int64         
 6   FTAG      42593 non-null  int64         
 7   FTR       42593 non-null  object        
 8   Target    42593 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 2.9+ MB


Let's create the target variable, which will equal to 1 for matches ending with 3 or more goals and 0 for the ones ending with 2 or less.

In [45]:
limited_data["Target"] = [1 if i > 2.5 else 0 for i in np.add(limited_data["FTHG"],limited_data["FTAG"])]
limited_data.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Target
0,B1,2019-07-26,19:30:00,Genk,Kortrijk,2,1,H,1
1,B1,2019-07-27,17:00:00,Cercle Brugge,Standard,0,2,A,0
2,B1,2019-07-27,19:00:00,St Truiden,Mouscron,0,1,A,0
3,B1,2019-07-27,19:00:00,Waregem,Mechelen,0,2,A,0
4,B1,2019-07-27,19:30:00,Waasland-Beveren,Club Brugge,1,3,A,1
